# Lemmatize Text From Twitter

## Step 1: Import libraries

In [1]:
import tensorflow as tf
import torch
import malaya
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import simplemma
import string
import re
import snscrape.modules.twitter as sntwitter
import pandas as pd
from cleantext import clean #to remove emoji in text
from googletrans import Translator

c:\Users\user\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
c:\Users\user\anaconda3\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


## Step 2: Scraping

In [2]:
# Created a list to append all tweet attributes(data)
tweets_content = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Anwar Ibrahim').get_items()):
    if i>10:
        break
    tweets_content.append(tweet.content)
tweets_content

['Jai Ho.\n#AnwarIbrahim #AmitabhBachchan https://t.co/V4l53tCRyF',
 '@Artashir_ @salehudin_sdb @FadhlinaSiddiq @FadhlinaComms @KemPendidikan @anwaribrahim Muka sama kan? 😂😂😂',
 'Dengan presiden negara jiran pun, @MuhyiddinYassin tpaksa perkenalkan diri😄..ni rakyat India biasa pun kenal @anwaribrahim.. https://t.co/5hMNzXRFwu',
 '@najib_bakar_ @anwaribrahim arey, bo lo na? meh apna pas ek sawal puchu',
 'My humble request to our PM @anwaribrahim Our history should be preserved, not buried by extremists!\n #AnwarIbrahim #kedah #bujangvalley https://t.co/7RKvdKw9XC',
 '@najib_bakar_ @anwaribrahim https://t.co/AkXGdAY79s',
 '@thefnma132 @Khir_Arif_Kay @anwaribrahim @chedetofficial @fmtoday Nazri &amp; Atok - should focus on ensuring this gov remains strong &amp; Malaysia prospers. If u can’t give constructive criticism, just don’t say anything. Let DSAI lead the country without your interference pls. We trust DSAI more than either one of u and/or a combination of both of u',
 'ANWAR IBRAH

## Step 3: Preprocessing

In [3]:
content_filtered = [] #cleaned
token_raw = []
tokenizer = malaya.tokenizer.Tokenizer()

def append_token(token):
    for i in token:
        token_raw.append(i)

for i in tweets_content:
    append_token(tokenizer.tokenize(i))
        
for i in token_raw:
    #Covert all letters to lowercase
    i.lower()
    #Remove emoji in text
    i = clean(i, no_emoji=True)
    #Remove all @mention,newlines,hyperlink
    i = re.sub("#[A-Za-z0-9_]+|@[A-Za-z0-9_]+|\n|http\S+|\"","",i)
    #Remove all white space
    #i = i.strip()
    #Remove punctuation
    i = i.translate(str.maketrans("", "", string.punctuation))
    #Remove no tweet content from list
    if i == "":
        continue
    else:
        content_filtered.append(i)

content_filtered

['jai',
 'ho',
 'muka',
 'sama',
 'kan',
 'dengan',
 'presiden',
 'negara',
 'jiran',
 'pun',
 'tpaksa',
 'perkenalkan',
 'diri',
 'ni',
 'rakyat',
 'india',
 'biasa',
 'pun',
 'kenal',
 'arey',
 'bo',
 'lo',
 'na',
 'meh',
 'apna',
 'pas',
 'ek',
 'sawal',
 'puchu',
 'my',
 'humble',
 'request',
 'to',
 'our',
 'pm',
 'our',
 'history',
 'should',
 'be',
 'preserved',
 'not',
 'buried',
 'by',
 'extremists',
 'nazri',
 'atok',
 'should',
 'focus',
 'on',
 'ensuring',
 'this',
 'gov',
 'remains',
 'strong',
 'malaysia',
 'prospers',
 'if',
 'u',
 'can',
 't',
 'give',
 'constructive',
 'criticism',
 'just',
 'don',
 't',
 'say',
 'anything',
 'let',
 'dsai',
 'lead',
 'the',
 'country',
 'without',
 'your',
 'interference',
 'pls',
 'we',
 'trust',
 'dsai',
 'more',
 'than',
 'either',
 'one',
 'of',
 'u',
 'and',
 'or',
 'a',
 'combination',
 'of',
 'both',
 'of',
 'u',
 'anwar',
 'ibrahim',
 'dan',
 'zahid',
 'pembawa',
 'musibah',
 'dengan',
 'berlaku',
 'perpecahan',
 'antara',
 'k

## Step 4: Normalization

In [5]:
bm_dict = pd.read_csv('BM_dict.csv')
bm_dict

,rojak,actual
0,abeh,habis
1,abes,habis
2,abih,habis
3,abg,adik
4,ade,ada
...,...,...
595,ohh,rupanya
596,yerlah,betul
597,klau,kalau
598,xpenting,tidak penting


### First Normalize

In [6]:
content_normalized1 = []

for i in content_filtered:
    if (bm_dict['rojak'] == i).any():
        str(content_normalized1.append(bm_dict[bm_dict['rojak'] == i]['actual'].to_string(index = False)))
    else:
        content_normalized1.append(i)

content_normalized1

['jai',
 'ho',
 'muka',
 'sama',
 'kan',
 'dengan',
 'presiden',
 'negara',
 'jiran',
 'pun',
 'tpaksa',
 'perkenalkan',
 'diri',
 'ini',
 'rakyat',
 'india',
 'biasa',
 'pun',
 'kenal',
 'arey',
 'bo',
 'lo',
 'na',
 'meh',
 'apna',
 'pas',
 'ek',
 'sawal',
 'puchu',
 'my',
 'humble',
 'request',
 'to',
 'our',
 'perdana menteri',
 'our',
 'history',
 'should',
 'be',
 'preserved',
 'not',
 'buried',
 'by',
 'extremists',
 'nazri',
 'atok',
 'should',
 'focus',
 'on',
 'ensuring',
 'this',
 'gov',
 'remains',
 'strong',
 'malaysia',
 'prospers',
 'if',
 'u',
 'can',
 'nanti\nnanti',
 'give',
 'constructive',
 'criticism',
 'just',
 'don',
 'nanti\nnanti',
 'say',
 'anything',
 'let',
 'dato seri anwar ibrahim',
 'lead',
 'the',
 'country',
 'without',
 'your',
 'interference',
 'pls',
 'we',
 'trust',
 'dato seri anwar ibrahim',
 'more',
 'than',
 'either',
 'one',
 'of',
 'u',
 'and',
 'or',
 'a',
 'combination',
 'of',
 'both',
 'of',
 'u',
 'anwar',
 'ibrahim',
 'dan',
 'zahid',
 '

### Second Normalize 

In [7]:
corrector = malaya.spelling_correction.probability.load()
normalizer = malaya.normalize.normalizer(corrector)

content_normalized2 = []

for i in content_normalized1:
    content_normalized2.append(normalizer.normalize(i)["normalize"])
       
content_normalized2

['jadi',
 'hio',
 'muka',
 'sama',
 'kan',
 'dengan',
 'presiden',
 'negara',
 'jiran',
 'pun',
 'tpaksa',
 'perkenalkan',
 'diri',
 'ini',
 'rakyat',
 'india',
 'biasa',
 'pun',
 'kenal',
 'carey',
 'obo',
 'plo',
 'nya',
 'mehr',
 'pena',
 'pas',
 'kek',
 'syawal',
 'fuchu',
 'mayu',
 'humble',
 'request',
 'kepada',
 'our',
 'perdana menteri',
 'our',
 'history',
 'should',
 'jadi',
 'preserved',
 'not',
 'buried',
 'dengan',
 'extremists',
 'nazri',
 'tok',
 'should',
 'focus',
 'on',
 'ensuring',
 'this',
 'ogv',
 'remains',
 'strong',
 'malaysia',
 'prospers',
 'jika',
 'awak',
 'can',
 'nanti nanti',
 'give',
 'constructive',
 'criticism',
 'just',
 'don',
 'nanti nanti',
 'say',
 'anything',
 'let',
 'dato seri anwar ibrahim',
 'lead',
 'the',
 'country',
 'without',
 'your',
 'interference',
 'tolong',
 'wwe',
 'trust',
 'dato seri anwar ibrahim',
 'more',
 'than',
 'either',
 'one',
 'of',
 'awak',
 'and',
 'atau',
 'a',
 'combination',
 'of',
 'both',
 'of',
 'awak',
 'anwar

## Step 5: Translation

In [11]:
from googletrans import Translator
translator = Translator()

content_translated = []

for i in content_normalized2:
    translate = translator.translate(i, src='en', dest = 'id')
    content_translated.append(translate.text.lower())

content_translated

ConnectTimeout: _ssl.c:1112: The handshake operation timed out

## Step 6: Lemmatization

In [22]:
content_lemma = []
sastrawi = malaya.stem.sastrawi()

for i in content_translated:
    content_lemma.append(sastrawi.stem(i).lower())
       
content_lemma

['fha',
 'kuat',
 'siti',
 'parah',
 'suda',
 'ini',
 'jeng',
 'pun pakat harap parket nasional',
 'bagi',
 'gerak',
 'kerja',
 'shalat',
 'diam',
 'shalat',
 'apa',
 'boleh',
 'barat',
 'hendak',
 'suh',
 'anwar',
 'jadi',
 'pokok',
 'pisang',
 'macam',
 'abah',
 'dengan',
 'ayohchik',
 'dalam',
 'partimen',
 'agak',
 'kah',
 'kejap',
 'yang',
 '33',
 'bulan',
 'itu',
 'selesai',
 'tidak',
 'bodoh',
 'kamu',
 'jawab',
 'aku',
 'sekarang',
 'jangan',
 'utama',
 'babi',
 'kalau',
 'sederhana',
 'kenapa',
 'tidak',
 'pernah',
 'selesai',
 'wulan',
 'bodoh',
 'saya',
 'tahu',
 'itu',
 'tetapi',
 'kadap-karang',
 'yang',
 'ekstremis',
 'ini',
 'mana',
 'fikir',
 'tahu',
 'dia',
 'sahaja',
 'betul',
 '3',
 'entit',
 'ini',
 'tuhan',
 'yang',
 'sama',
 'cuma',
 'cara',
 'berbeza',
 'cara',
 'berbeza',
 'juga',
 'fahaman',
 'berbeza',
 'rakyat',
 'lagi',
 'risau',
 'tidak',
 'yakin',
 'take upm',
 'yang',
 'singkir',
 'reja',
 'sendiri',
 'dengan',
 'langkah',
 'sheraton',
 'rehat',
 'tong be

## Output Result

In [24]:
raw_lemma_df = pd.DataFrame(content_filtered)
norma_df = pd.DataFrame(content_normalized2)
trans_df = pd.DataFrame(content_translated)
lemma_df = pd.DataFrame(content_lemma)

raw_lemma_df.rename(columns = {0:'Raw'}, inplace = True)
raw_lemma_df['Normalized'] = norma_df
raw_lemma_df['Translated'] = trans_df
raw_lemma_df['Lemmatized'] = lemma_df

raw_lemma_df.sample(50)

,Raw,Normalized,Translated,Lemmatized
6,geng,geng,jeng,jeng
73,sama,sama,sama,sama
227,stuju,setuju,setuju,tuju
201,madani,madani,madani,madani
151,panggil,panggil,panggil,panggil
142,telur,telur,telur,telur
129,la,lah,lah,lah
221,mtk,meminta,meminta,minta
64,tau,tahu,tahu,tahu
219,boleh,boleh,boleh,boleh


# Lemmatize From User Input

In [ ]:
def lemmatization_input(userinput):
    token_raw = []
    content_filtered = []
    content_normalized1 = []
    content_normalized2 = []
    content_translated = []
    content_lemma = []
    tokenizer = malaya.tokenizer.Tokenizer()
    corrector = malaya.spelling_correction.probability.load()
    normalizer = malaya.normalize.normalizer(corrector)
    translator = Translator()
    sastrawi = malaya.stem.sastrawi()
    bm_dict = pd.read_csv('BM dict.csv')

    
    def chk_content(token_raw,content_filtered):
        if len(token_raw) != 0:
            token_raw.clear()
            content_filtered.clear()
            content_normalized1.clear()
            content_normalized2.clear()
            content_translated.clear()
            content_lemma.clear()
    
    def append_token(token):
        for i in token:
            token_raw.append(i)
    
    chk_content(token_raw,content_filtered)
    append_token(tokenizer.tokenize(userinput))

    for i in token_raw:
        #Covert all letters to lowercase
        i.lower()
        #Remove emoji in text
        i = clean(i, no_emoji=True)
        #Remove all @mention,newlines,hyperlink
        i = re.sub("#[A-Za-z0-9_]+|@[A-Za-z0-9_]+|\n|http\S+|\"","",i)
        #Remove punctuation
        i = i.translate(str.maketrans("", "", string.punctuation))
        if i == "":
            continue
        else:
            content_filtered.append(i)
    
    for i in content_filtered:
        if (bm_dict['rojak'] == i).any():
            str(content_normalized1.append(bm_dict[bm_dict['rojak'] == i]['actual'].to_string(index = False)))
        else:
            content_normalized1.append(i)
    
    for i in content_normalized1:
        content_normalized2.append(normalizer.normalize(i)["normalize"])
        
    for i in content_normalized2:
        translate = translator.translate(i, src='en', dest = 'id')
        content_translated.append(translate.text)
    
    for i in content_translated:
        content_lemma.append(sastrawi.stem(i))

    return content_lemma

In [ ]:
userinput = input("Please enter sentence: ")
lemmatization_input(userinput)

In [13]:
translate = translator.translate('tidak', src='en', dest = 'id')
translate.text

'Tidak'